In [1]:
#Inicializacion de spark session y creacion de nuestra instancia
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StructType,IntegerType,StringType,FloatType

spark = SparkSession.builder.appName('Sesion3_joins').getOrCreate()

In [18]:
#Vamos a crear un dataframe de empleados, que nos va a valer para el ejercicio de practica que leugo comentaremos..
empleados = [(1,"Jaime",-1,"2018","10","M",3000), \
    (2,"Rosa",1,"2010","20","F",4000), \
    (3,"Maria",1,"2010","10","F",1000), \
    (4,"Vanesa",2,"2005","10","F",2000), \
    (5,"Leonardo",2,"2010","40","M",-1), \
      (6,"Paula",2,"2013","50","F",-1) \
  ]
empColumns = ["emp_id","nombre","superior_emp_id","fecha_inicio","emp_dept_id","genero","sueldo"]

In [19]:
empleados_df = spark.createDataFrame(data=empleados, schema = empColumns)
empleados_df.printSchema()
empleados_df.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- fecha_inicio: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- sueldo: long (nullable = true)

+------+--------+---------------+------------+-----------+------+------+
|emp_id|nombre  |superior_emp_id|fecha_inicio|emp_dept_id|genero|sueldo|
+------+--------+---------------+------------+-----------+------+------+
|1     |Jaime   |-1             |2018        |10         |M     |3000  |
|2     |Rosa    |1              |2010        |20         |F     |4000  |
|3     |Maria   |1              |2010        |10         |F     |1000  |
|4     |Vanesa  |2              |2005        |10         |F     |2000  |
|5     |Leonardo|2              |2010        |40         |M     |-1    |
|6     |Paula   |2              |2013        |50         |F     |-1    |
+------+--------+---------------+------------+-----------

In [20]:
departamento = [("Financiero",10),("Marketing",20),("Ventas",30),("Desarrollo",40)]
deptColumns = ["dept_nombre","dept_id"]
departamento_df = spark.createDataFrame(data=departamento, schema = deptColumns)
departamento_df.printSchema()
departamento_df.show(truncate=False)

root
 |-- dept_nombre: string (nullable = true)
 |-- dept_id: long (nullable = true)

+-----------+-------+
|dept_nombre|dept_id|
+-----------+-------+
|Financiero |10     |
|Marketing  |20     |
|Ventas     |30     |
|Desarrollo |40     |
+-----------+-------+



In [21]:
#Hacemos un inner join por departamento
empleados_df.join(departamento_df,empleados_df.emp_dept_id ==  departamento_df.dept_id,"inner").show(truncate=False)

+------+--------+---------------+------------+-----------+------+------+-----------+-------+
|emp_id|nombre  |superior_emp_id|fecha_inicio|emp_dept_id|genero|sueldo|dept_nombre|dept_id|
+------+--------+---------------+------------+-----------+------+------+-----------+-------+
|1     |Jaime   |-1             |2018        |10         |M     |3000  |Financiero |10     |
|3     |Maria   |1              |2010        |10         |F     |1000  |Financiero |10     |
|4     |Vanesa  |2              |2005        |10         |F     |2000  |Financiero |10     |
|2     |Rosa    |1              |2010        |20         |F     |4000  |Marketing  |20     |
|5     |Leonardo|2              |2010        |40         |M     |-1    |Desarrollo |40     |
+------+--------+---------------+------------+-----------+------+------+-----------+-------+



In [22]:
#LEFTSEMI nos va a devolver lo que coincide (como un inner) pero solo las columnas del DF de la izquerda
empleados_df.join(departamento_df,empleados_df.emp_dept_id ==  departamento_df.dept_id,"LEFTSEMI").show(truncate=False)

+------+--------+---------------+------------+-----------+------+------+
|emp_id|nombre  |superior_emp_id|fecha_inicio|emp_dept_id|genero|sueldo|
+------+--------+---------------+------------+-----------+------+------+
|1     |Jaime   |-1             |2018        |10         |M     |3000  |
|3     |Maria   |1              |2010        |10         |F     |1000  |
|4     |Vanesa  |2              |2005        |10         |F     |2000  |
|2     |Rosa    |1              |2010        |20         |F     |4000  |
|5     |Leonardo|2              |2010        |40         |M     |-1    |
+------+--------+---------------+------------+-----------+------+------+



In [23]:
#LEFTANTI nos va a devolver lo que NO coincide pero solo las columnas del DF de la izquerda

empleados_df.join(departamento_df,empleados_df.emp_dept_id ==  departamento_df.dept_id,"LEFTANTI").show(truncate=False)

+------+------+---------------+------------+-----------+------+------+
|emp_id|nombre|superior_emp_id|fecha_inicio|emp_dept_id|genero|sueldo|
+------+------+---------------+------------+-----------+------+------+
|6     |Paula |2              |2013        |50         |F     |-1    |
+------+------+---------------+------------+-----------+------+------+



In [25]:
#Podemos elegir que solo nos saque un campo por ejemplo:
empleados_df.join(departamento_df,empleados_df.emp_dept_id ==  departamento_df.dept_id,"LEFTANTI").select('nombre').show()

+------+
|nombre|
+------+
| Paula|
+------+



In [29]:
#Comprobamos que RIGHT RIGHTOUTER dan el mismo resultado 
empleados_df.join(departamento_df,empleados_df.emp_dept_id ==  departamento_df.dept_id,"RIGHT").show()
empleados_df.join(departamento_df,empleados_df.emp_dept_id ==  departamento_df.dept_id,"RIGHTOUTER").show()

+------+--------+---------------+------------+-----------+------+------+-----------+-------+
|emp_id|  nombre|superior_emp_id|fecha_inicio|emp_dept_id|genero|sueldo|dept_nombre|dept_id|
+------+--------+---------------+------------+-----------+------+------+-----------+-------+
|     1|   Jaime|             -1|        2018|         10|     M|  3000| Financiero|     10|
|     3|   Maria|              1|        2010|         10|     F|  1000| Financiero|     10|
|     4|  Vanesa|              2|        2005|         10|     F|  2000| Financiero|     10|
|  null|    null|           null|        null|       null|  null|  null|     Ventas|     30|
|     2|    Rosa|              1|        2010|         20|     F|  4000|  Marketing|     20|
|     5|Leonardo|              2|        2010|         40|     M|    -1| Desarrollo|     40|
+------+--------+---------------+------------+-----------+------+------+-----------+-------+

+------+--------+---------------+------------+-----------+------+----